In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/thiesen/Documents/Cleantech_Concepts/result_work_text.csv')

/tmp/ipykernel_3296600/3664583072.py:1: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/thiesen/Documents/Cleantech_Concepts/result_work_text.csv')


In [10]:
len(df)

124877